Caixeiro Viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.

O código abaixo é um gerador de coordenadas de cidades. Use ele caso queira.



<hr>

## Importações



In [1]:
import random
from itertools import permutations # itertools = método que cria 'iterators' para loop eficiente 

# funções podem ser encontradas no arquivo 'funcoes.py'
from funcoes import cria_cidades
from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import funcao_objetivo_cv
from funcoes import selecao_torneio_min as funcao_selecao
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

## Códigos e discussão



In [32]:
# problema do caixeiro viajante: nosso personagem viagem muito, mas ele quer retornar para casa
# problema de minimização: ele quer o menor caminho para retornar
# problema NP: só conseguimos saber se uma resposta é melhor se testarmos todas as opções 

# CONSTANTES 
# interessante escrever as constantes com letra miúscula, permite diferenciá-las dentro da função 

# relacionadas à busca 
## podem ser alteradas
TAMANHO_POP = 25 # número total de indivíduos na população 
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3 # utilizando torneio para selecionar indivíduos, minimização = menor fitness passa

# relacionadas ao problema a ser resolvido
## não podem ser alteradas
NUMERO_DE_CIDADES = 5 # muito importante
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

In [33]:
# funções locais
# pensadas para esse problema em específico, não serão encontradas no arquivo 'funcoes.py'
# podem ser funções parciais, ou seja, funções que apenas chamam outras funções

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv(individuo, CIDADES)

In [34]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES) 
# criação consiste no agrupamento de todos os indivíduos = n° máx dado pela constante TAMANHO_POP definida anteriormente
# cada indivíduo, por sua vez, é formado por um número definido de genes (sendo as cidades as correspondentes aos genes)

melhor_fitness_ja_visto = float("inf")  # 'inf' é a notação de infinito
                                        # float permite trabalhar com números decimais
                                        # logo, o melhor menor fitness pode ser um número qualquer, incluindo decimais

# critério de parada, precisamos testar todas as opções, então precisamos passar por todas as gerações
for n in range(NUM_GERACOES): # para cada geração dentro do número total de gerações
    
    # seleção (primeira parte, escolhendo os pais e mães)
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # segunda parte, cruzamento
    # novamente, dentro da população inteira, alternaremos os indivíduos entre pai e mãe
    # quer dizer que, se o pai for o elemento na posição 0, a mãe será 1, o próximo pai 2 e assim em diante, até o último indiv.
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0 # contador é uma subclasse de dicionários
    
    for pai, mae in zip(pais, maes): # mexe-se simultaneamente nas listas de 'pais' e 'maes'
        if random.random() <= CHANCE_CRUZAMENTO: 
            filho1, filho2 = funcao_cruzamento(pai, mae) # lembrando que, aqui, o cruzamento é feito a partir de dois cortes
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    # a mutação aqui, esolhe dois genes e troca eles entre si
    # exemplo: se a ordem era [a, b, c, d, e] e ele escolher 'c' e 'd', a ordem passará a ser [a, b, d, c, e]
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n] #individuos como elementos posicionados na lista de populacao
            populacao[n] = funcao_mutacao(individuo) # o individuo selecionado aleatoriamente é submetido à mutação        
            
    # melhor individuo já visto até agora
    # garantimos que o melhor fitness vai sendo atualizado conforme novas análises
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:       
        posicao = fitness.index(menor_fitness) # index retorna a posição
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness 
        
    print(populacao[0])

['Cidade 1', 'Cidade 4', 'Cidade 3', 'Cidade 2', 'Cidade 0']
['Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 0', 'Cidade 4']
['Cidade 3', 'Cidade 1', 'Cidade 2', 'Cidade 0', 'Cidade 4']
['Cidade 1', 'Cidade 4', 'Cidade 0', 'Cidade 2', 'Cidade 3']
['Cidade 0', 'Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 4']
['Cidade 1', 'Cidade 0', 'Cidade 2', 'Cidade 3', 'Cidade 4']
['Cidade 1', 'Cidade 2', 'Cidade 0', 'Cidade 3', 'Cidade 4']
['Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 0', 'Cidade 4']
['Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 0', 'Cidade 4']
['Cidade 4', 'Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 0']
['Cidade 1', 'Cidade 4', 'Cidade 3', 'Cidade 2', 'Cidade 0']
['Cidade 1', 'Cidade 4', 'Cidade 3', 'Cidade 2', 'Cidade 0']
['Cidade 4', 'Cidade 3', 'Cidade 1', 'Cidade 2', 'Cidade 0']
['Cidade 4', 'Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 0']
['Cidade 3', 'Cidade 4', 'Cidade 1', 'Cidade 2', 'Cidade 0']
['Cidade 2', 'Cidade 4', 'Cidade 1', 'Cidade 3', 'Cidade 0']
['Cidade 4', 'Cidade 1',

In [35]:
melhor_fitness_ever = float("inf") # novamente, 'inf' de infinito

# testando todas as permutações possíveis em busca da resposta (cidade) com menor fitness e menor distância
for caminho in permutations(list(CIDADES.keys())): 
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

In [36]:
# Checando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com distância:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)


Melhor individuo obtido por algoritmos genéticos:
['Cidade 4', 'Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 0'] com distância: 1.84166565910256

Melhor individuo obtido por busca exaustiva:
('Cidade 4', 'Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 0') com distância: 1.84166565910256


<br>

<br>

#### DISCUSSÃO

<div style=' text-align: justify; text-justify: inter-word;'>
    <br> O objetivo do presente experimento consistia em encontrar uma resposta que permitisse que um caixeiro viajante pudesse viajar entre diferentes cidades, retornando no final para a cidade inicial e, tudo isso, fazendo o menor caminho possível. Dessa maneira, já que queremos o menor valor possível, concluímos desde o início também que se trata de um problema de minimização. Por conta disso, pode-se observar no decorrer do código que os melhores fitness são aqueles que apresentam valores inferiores em comparação aos outros gerados.
    <br> Para atingir tal objetivo fazendo uso de algoritmos genéticos, foi necessário, primeiramente, que criassemos a nossa população estudada (de 25 indivíduos), ou seja, era preciso que o computador gerasse as possibilidades de caminhos pelos quais o viajante passaria com base nas constantes definidas logo no início do problema. Cada possível caminho corresponde a um indivíduo e cada indivíduo é formado por um número específico (5, definido nas constantes) de cidades, que, por sua vez, corresponderiam aos genes. Trabalhamos, em sala de aula, com mil gerações, o que implica em 1000 populações diferentes entre si terem sido criadas, uma sendo resultado do cruzamente entre dois indivíduos (com exceção da primeira). Esses indivíduos foram cruzados com a metologia ordenada, pela qual a prole mantém os mesmos genes dos pais (mesmas cidades), mas há uma alteração na disposição desses por conta dos dois cortes que são feitos. Depois que os novos indivíduos de uma nova população em uma nova geração foram criados, eles ainda são submetidos à mutações (lembrando que a chance real de ocorrer uma mutação foi definida como 0.05 nas constantes). Essa mutação é feita simplesmente alterando a ordem dos genes (das cidades) nos indivíduos. Dados os passos iniciais, inicia-se o momento no qual os fitness são avaliados tanto para as populações, quanto para os indivíduos, lembrando que se está em busca do menor valor gerado. No fim, escolhido o indivíduo com o menor valor de fitness, sabe-se consequentemente também qual é a reposta do problema e qual a menor distância que o caixeiro precisará percorrer.
    <br> Ao rodar o código pela primeira vez, o valor de menor distância encontrado foi de 2.2506 (para uma população de tamanho 25). Na segunda vez, tendo aumentado o tamanho da população (= 30) e mantendo os valores de todas as outras constantes, o valor caiu para 2.0063. Na terceira vez, ao aumentar o tamanho para 35, o valor da distância aumentou para 2.4987, o maior que tivemos até agora. Na quarta e quinta vez, resolvi diminuir a população inicial (25) em duas vezes, testando então o tamanho igual a 20 e 15. As distâncias encontradas foram, respectivamente, 1.8850 e 2.2842. Por fim, retornando a 25, o valor de 1.8417 foi encontrado. Essas alterações em somente um dos parâmetros me mostraram que o resultado final para ter um caráter aleatório, já que mesmo retornando à população inicial o valor obtido foi outro. Então, foi como se, a cada vez que eu rodava o código, um novo indivíduo era escolhido, configurando uma nova opção de caminho para o caixeiro na nova situação que foi criada.
</div>

 A conclusão é um texto breve refletindo sobre a hipótese / objetivo, o que foi feito, e onde chegamos.


<hr>

## Conclusão
> <div style=' text-align: justify; text-justify: inter-word;'> Resumidamente, o experimento do caixeiro viajante se mostrou um pouco mais desafiador do que os anteriores. Dessa vez, parecia que tínhamos mais limitações para levar em conta. As próprias condições de que o viajante não poderia passar mais de uma vez pela mesma cidade e de que ele precisava retornar para a cidade inicial por si só já pareciam difíceis de serem traduzidas em uma linguagem que fosse compredida pelo algoritmo genético. Contudo, o código elaborado pelo professor Daniel e utilizado aqui nesse notebook foi bastante claro e nos deu o que o problema solicitava: o caminho com a menor distância.
</div>

> <div style=' text-align: justify; text-justify: inter-word;'> Por último, algo que me chamou a atenção foi a utilização de uma função que foi escrita, na realidade, de maneira mais tradicional, matemática e, menos nos padrões de algoritmos geéticos: a função da distância entre dois pontos. Sim, posteriormente ela foi incluida na função que buscava o fitness, por exemplo, mas a maneira que ela foi escrita me lembrou mais dos esforços iniciais de aprendizado do Python. Achei interessante ver essas ferramentas sendo utilizadas conjuntamente, uma dentro da outra. Saber que isso é uma possibilidade facilita a pensar na reoslução de outros problemas.</div>



<hr>

## Playground



**Anotações da aula**
<br> no cruzamento, pega o que está dentro do corte do pai para o primeiro filho
<br> no cruzamento, pega o que está dentro do corte da mãe para o segundo filho 
<br> - sobre o começo do corte
<br> cortes são escolhidos aleatoriamente
<br> seleção não altera o indivíduo, então podemos usar ele
<br> lista envolvem ordem e ela pode ser alterada
<br> tupla também envolve ordem, mas ela é imutável > as coordenadas aqui são tuplas (cidade não muda de lugar)